In [ ]:
import numpy as np
import pandas as pd
import os
import sys

In [ ]:
data_dir = "/Users/roopal/workspace/kaggle/quora_question_pairs/data"

In [ ]:
train_file = os.path.join(data_dir, "train.csv")
test_file = os.path.join(data_dir, "test.csv")
sample_submission = os.path.join(data_dir, "sample_submission.csv")

In [ ]:
df_train = pd.read_csv(train_file, index_col=False)

In [ ]:
df_train = df_train.dropna()

In [ ]:
df_train

In [ ]:
REPO_LOCAL_PATH = "/Users/roopal/ggrepo/ds-tws-backend"

In [ ]:
if REPO_LOCAL_PATH not in sys.path:
    sys.path.append(REPO_LOCAL_PATH)
    print "repo local path added"
else:
    print "repo local path already added"

In [ ]:
from ipywidgets import FloatProgress
from IPython.display import display

In [ ]:
MODEL = "qoura_qp"

In [ ]:
from util.util_text_process import get_sentence_tokenizer, get_sentences, tokenize as tokenize_en
from deep_learning.embeddings.GloveEmbeddings import GloveEmbeddings
from deep_learning.corpus_reader.CorpusReader import CorpusReader

In [ ]:
WORD2IDX, _ = GloveEmbeddings.get_embeddings_with_custom_tokens()
len(WORD2IDX)

In [ ]:
EN_SENT_TOKENIZER = get_sentence_tokenizer()

In [ ]:
def process_sentences(sentences, word2idx, build_word2idx=False):
    list_line_word_idx = list()
    for sentence in sentences:
        sentence = CorpusReader.clean_line(sentence)
        
        words = tokenize_en(sentence)
        
        line_word_idx = list()
        for word in words:
            idx = CorpusReader.get_word_index(word, word2idx, build_word2idx)
            line_word_idx.append(idx)
        list_line_word_idx.append(line_word_idx)
        
    return list_line_word_idx

In [ ]:
def get_question_sent_idx(question1):
    try:
        q1 = process_sentences([question1], WORD2IDX)
    except Exception as e:
        print str(e), question1
        q1 = [[]]
        
#     try:
#         q2 = process_sentences([question2], WORD2IDX)
#     except Exception as e:
#         print str(e), question2
#         q2 = []
#     return id, q1, q2
    return q1

In [ ]:
df_train["q1_sent_idx"] = df_train["question1"].apply(lambda x: get_question_sent_idx(x))

In [ ]:
df_train["q2_sent_idx"] = df_train["question2"].apply(lambda x: get_question_sent_idx(x))

In [ ]:
df_train

In [ ]:
df_train[df_train["qid2"]==16064]

In [ ]:
df_train.groupby(["is_duplicate"]).agg(["count"])

In [ ]:
df_train.to_pickle(train_file+".pkl")

In [ ]:
df_test = pd.read_csv(test_file, index_col=False)

In [ ]:
df_test["q1_sent_idx"] = df_test["question1"].apply(lambda x: get_question_sent_idx(x))

In [ ]:
df_test["q2_sent_idx"] = df_test["question2"].apply(lambda x: get_question_sent_idx(x))

In [ ]:
df_test

In [ ]:
df_test.to_pickle(test_file+".pkl")

In [ ]:
df_test.columns

In [ ]:
len(df_test)

In [ ]:
for item, frame in df_test["q1_sent_idx"].iteritems():
    if len(frame) == 1 and len(frame[0]) > 0:
        continue
    print item, frame

In [ ]:
for item, frame in df_test["q2_sent_idx"].iteritems():
    if len(frame) == 1 and len(frame[0]) > 0:
        continue
    print item, frame

In [ ]:
print "done"